In [1]:
#%pip install agentpy
#%pip install owlready2

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/27.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/27.3 MB ? eta -:--:--
     - -------------------------------------- 0.8/27.3 MB 3.3 MB/s eta 0:00:08
     - -------------------------------------- 1.3/27.3 MB 2.8 MB/s eta 0:00:10
     -- ------------------------------------- 1.8/27.3 MB 2.6 MB/s eta 0:00:10
     --- ------------------------------------ 2.4/27.3 MB 2.7 MB/s eta 0:00:10
     ---- ----------------------------------- 3.1/27.3 MB 2.8 MB/s eta 0:00:09
     ----- ---------------------------------- 3.7/27.3 MB 2.8 MB/s eta 0:00:09
     ------ --------------------------------- 4.2/27.3 MB 2.7 MB/s eta 0:00:09
     ------ --------------------------------- 4.7/27.3 MB 2.8 MB/s eta 0:00:08
  

In [ ]:
from owlready2 import *
# Model design
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns

In [ ]:
onto = get_ontology("file:///content/wealth_ontology.owl").load()

In [ ]:
#Tienda

with onto: 
    class Agent(Thing):
        pass
    class TiendaAgente(Agent):
        pass
    class PedidosEnTienda(Thing):
        pass

In [ ]:
class Message():
    environment_buffer = []
    def __init__(self,sender=None,receiver=None,performative=None,content=None):
        self.sender = sender
        self.receiver = receiver
        self.performative = performative
        self.content = content
    def __str__(self):
        return f"\n\
        Sender: {self.sender}, \n\
        Receiver: {self.receiver}, \n\
        Performative: {self.performative}, \n\
        Content: {self.content}"
    def send(self):
        """
        The send function is used to send a message to the environment buffer.
        """
        Message.environment_buffer.append(self)
